# TrOCR: Analiza i Ulepszenia

## Wprowadzenie

W ramach tego eksperymentu pracowałem nad modelem **TrOCR**, próbując nauczyć go odczytywania pisma odręcznego. Pierwsza wersja kodu implementowała podstawowe podejście, natomiast druga została ulepszona o kilka technik optymalizacyjnych.

Pomimo użycia wysokiej jakości danych, **trening trwał bardzo długo**, a wyniki, choć nieco lepsze po wprowadzeniu optymalizacji, nadal nie były satysfakcjonujące. W związku z tym zdecydowałem się **zmienić kierunek prac** – zamiast kontynuować rozwijanie tego modelu, skupię się na **kolejnym kroku**, czyli **analizie i uzupełnianiu już odczytanych danych**.

---

## Struktura projektu
W tym notebooku omówimy:
1. **Pierwszą wersję kodu (`train_trocr.py`)** – podstawowy model TrOCR bez dodatkowych optymalizacji.
2. **Drugą wersję kodu (`train_trocr2.py`)** – ulepszony model z regularyzacją, mieszanym treningiem na GPU i wersjonowaniem wyników.
3. **Porównanie wyników** obu podejść oraz ich wpływ na proces treningu.
4. **Wnioski** na temat trudności w trenowaniu modeli OCR dla pisma odręcznego i dalszych kroków w projekcie.

---

## 1. Pierwsza wersja kodu (`train_trocr.py`)

Plik: **train_trocr.py**

### Główne elementy:
- **Załadunek modelu i procesora**:



In [ ]:
processor = TrOCRProcessor.from_pretrained("microsoft/trocr-base-stage1")
model = VisionEncoderDecoderModel.from_pretrained("microsoft/trocr-base-stage1")

- **Optymalizator bez regularyzacji:**

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

- **Proces treningowy:**

In [ ]:
for epoch in range(num_epochs):
    train_loss = train_epoch(model, train_loader, optimizer, device, epoch)
    val_loss = validate_epoch(model, val_loader, device, epoch)
    print(f"Epoch {epoch+1} - Train Loss: {train_loss:.4f} - Val Loss: {val_loss:.4f}")


- **Problemy:**

    - ***Brak regularyzacji (weight_decay) – może prowadzić do przeuczenia modelu.***

    - ***Trening w pełnej precyzji (float32), co może być wolniejsze na GPU.***

    - ***Brak automatycznego zapisu wersji modelu po każdej epoce.***

---

## 2. Ulepszona wersja kodu (train_trocr2.py)
Plik: train_trocr2.py

## Ulepszenia:
   - ***Dodanie regularyzacji weight_decay***
      - Zapobiega przeuczeniu modelu.

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5, weight_decay=1e-2)


- ***Zastosowanie mieszanej precyzji (float16) dla szybszego treningu na GPU***
    - Skraca czas trenowania i zmniejsza zużycie pamięci GPU.

In [ ]:
from torch.amp import GradScaler, autocast

scaler = GradScaler()
with autocast(device_type='cuda'):
    outputs = model(pixel_values=images, labels=labels)
    loss = outputs.loss


---
## 3. Przeprowadzanie testów
Plik: test_trocr.py

Po zakończeniu treningu modelu przeprowadziłem testy na wcześniej przygotowanym zestawie danych. Testy miały na celu sprawdzenie, jak dobrze model radzi sobie z przewidywaniem tekstu na nowych obrazach z pismem odręcznym.

## Jak wyglądał proces testowania?
***1. Załadowanie modelu i danych testowych:***

Model został wczytany z zapisanej wersji (../models/trocr*).

Zestaw testowy zawierał obrazy linii tekstu oraz odpowiadające im etykiety (data/test/lines/ oraz data/processed/labels.json).

***2. Przetwarzanie obrazów i tokenizacja etykiet***

***3. Generowanie przewidywań przez model***

***4. Wyniki były porównywane z rzeczywistymi etykietami (ground_truths vs predictions)***

***5. Zapisywanie wyników do pliku:***

Wszystkie przewidywania wraz z rzeczywistymi etykietami zostały zapisane w test_results*.txt, co pozwoliło na późniejszą analizę błędów.

---
## 4. Porównanie wyników

- Czy ulepszenia poprawiły wyniki?

Nie było dużej poprawy – mimo zmian model nadal miał trudności z dokładnym rozpoznawaniem pisma odręcznego.

Problem generalizacji – model działał lepiej na danych treningowych, ale miał trudności na nowych przykładach.

 - Dlaczego wyniki są wciąż nieidealne?

TrOCR wymaga bardzo dużych ilości wysokiej jakości danych, których nie zawsze można łatwo znaleźć.

Trening pisma odręcznego jest znacznie trudniejszy niż drukowanego tekstu – zmienność stylów pisania utrudnia jednoznaczne rozpoznanie.

Czas treningu był bardzo długi, a efekty nie były proporcjonalne do włożonego wysiłku.

## 5. Wnioski i dalsze kroki

- Podsumowanie:
TrOCR może być skuteczny, ale wymaga znacznie większej ilości danych i dłuższego czasu treningu.

Optymalizacje pomogły poprawić wydajność, ale nie rozwiązały fundamentalnych problemów modelu.

Model nadal ma trudności z precyzyjnym rozpoznawaniem pisma odręcznego, szczególnie jeśli styl jest nieregularny.

- Co dalej?

Z uwagi na ograniczenia i trudność dalszego rozwijania tego modelu, na tym etapie porzucam dalsze próby jego ulepszania. Kolejnym krokiem będzie: Analiza oraz uzupełnienie już odczytanych danych, aby poprawić jakość istniejących wyników zamiast próbować ulepszać model OCR.